In [ ]:
pip install distfit

In [ ]:
pip install pymc3

In [ ]:
# This block needs to be executed only on google colab

from google.colab import files
uploaded = files.upload()

In [6]:
# import the necessary libraries

import pandas as pd
import numpy as np
import pymc3 as pm
import random
import pprint
import matplotlib.pyplot as plt
from distfit import distfit
import scipy.integrate as integrate
import scipy.stats as stats
from scipy.special import gamma
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('alpha.csv')
pl = pd.read_csv('Master_players2.csv')
trf = pd.read_csv('no_duplicates_transfer.csv')

# Initial boxplots of player rating, from team rating, to team rating and fee (including outliers).

In [ ]:
plt.figure(figsize=(17,7))

plt.subplot(1,4,1)
plt.boxplot(df['RATING'])
plt.xlabel('Player rating', fontsize=15)
ratingq = np.quantile(df['RATING'], [0,0.25,0.5,0.75,1])

plt.subplot(1,4,2)
plt.boxplot(df[df['TT_RATING']!=0]['TT_RATING'])
plt.xlabel('To team rating', fontsize=15)
tteamq = np.quantile(df[df['TT_RATING']!=0]['TT_RATING'], [0,0.25,0.5,0.75,1])

plt.subplot(1,4,3)
plt.boxplot(df[df['FT_RATING']!=0]['FT_RATING'])
plt.xlabel('From team rating', fontsize=15)
fteamq = np.quantile(df[df['FT_RATING']!=0]['FT_RATING'], [0,0.25,0.5,0.75,1])

plt.subplot(1,4,4)
plt.boxplot(df['FEE'])
plt.xlabel('Transfer fee', fontsize=15)
plt.ylabel('Mil. euros', fontsize=15)
feeq = np.quantile(df['FEE'], [0,0.25,0.5,0.75,1])

plt.show()

# Remove outliers from player ratings and chunk of outliers from fees and boxplot of same as above.

In [10]:
def remove_outliers(df_in, col):
    Q1 = df_in[col].quantile(0.25)
    Q3 = df_in[col].quantile(0.75)
    IQR = Q3 - Q1
    if col == 'RATING':
        df_out = df_in.loc[(df_in[col] > Q1 - 1.5 * IQR) & (df_in[col] < Q3 + 1.5 *IQR)]
    if col == 'FEE':
        df_out = df_in.loc[(df_in[col] > Q1 - 1.5 * IQR) & (df_in[col] < Q3 + 4 *IQR)]

    # df_out = df_in.loc[(df_in[col] > Q1 - 1.5 * IQR) & (df_in[col] < Q3 + 1.5 *IQR)]
    return df_out

In [11]:
df = remove_outliers(df,'RATING')
df = remove_outliers(df,'FEE')
df.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(17,7))

plt.subplot(1,4,1)
plt.boxplot(df['RATING'])
plt.xlabel('Player rating', fontsize=15)
ratingq = np.quantile(df['RATING'], [0,0.25,0.5,0.75,1])

plt.subplot(1,4,2)
plt.boxplot(df[df['TT_RATING']!=0]['TT_RATING'])
plt.xlabel('To team rating', fontsize=15)
tteamq = np.quantile(df[df['TT_RATING']!=0]['TT_RATING'], [0,0.25,0.5,0.75,1])

plt.subplot(1,4,3)
plt.boxplot(df[df['FT_RATING']!=0]['FT_RATING'])
plt.xlabel('From team rating', fontsize=15)
fteamq = np.quantile(df[df['FT_RATING']!=0]['FT_RATING'], [0,0.25,0.5,0.75,1])

plt.subplot(1,4,4)
plt.boxplot(df['FEE'])
plt.xlabel('Transfer fee', fontsize=15)
plt.ylabel('Mil. euros', fontsize=15)
feeq = np.quantile(df['FEE'], [0,0.25,0.5,0.75,1])

plt.show()

# Associate player, from team and to team ratings and ages to classes

In [13]:
clss = [0]*len(df)
df['T_CLASS'] = clss
df['AGE_CLASS'] = clss
df['TF_CLASS'] = clss
df['P_CLASS'] = clss

for i in range(len(df)):
    if df['FT_RATING'].iloc[i] <= fteamq[1]:
        df['T_CLASS'].iloc[i] = 4
    if df['FT_RATING'].iloc[i] > fteamq[1] and df['FT_RATING'].iloc[i] <= fteamq[2]:
        df['T_CLASS'].iloc[i] = 3
    if df['FT_RATING'].iloc[i] > fteamq[2] and df['FT_RATING'].iloc[i] <= fteamq[3]:
        df['T_CLASS'].iloc[i] = 2
    if df['FT_RATING'].iloc[i] > fteamq[3]:
        df['T_CLASS'].iloc[i] = 1
    
    
    if df['AGE'].iloc[i] >= df['AGE'].min() and df['AGE'].iloc[i] <= 20:
        df['AGE_CLASS'].iloc[i] = 1
    if df['AGE'].iloc[i] > 20 and df['AGE'].iloc[i] <= 25:
        df['AGE_CLASS'].iloc[i] = 2
    if df['AGE'].iloc[i] > 25 and df['AGE'].iloc[i] <= 30:
        df['AGE_CLASS'].iloc[i] = 3
    if df['AGE'].iloc[i] > 30:
        df['AGE_CLASS'].iloc[i] = 4
        
    
    if df['FEE'].iloc[i] <= feeq[1]:
        df['TF_CLASS'].iloc[i] = 4
    if df['FEE'].iloc[i] > feeq[1] and df['FEE'].iloc[i] <= feeq[2]:
        df['TF_CLASS'].iloc[i] = 3
    if df['FEE'].iloc[i] > feeq[2] and df['FEE'].iloc[i] <= feeq[3]:
        df['TF_CLASS'].iloc[i] = 2
    if df['FEE'].iloc[i] > feeq[3]:
        df['TF_CLASS'].iloc[i] = 1
        
        
    if df['RATING'].iloc[i] <= ratingq[1]:
        df['P_CLASS'].iloc[i] = 4
    if df['RATING'].iloc[i] > ratingq[1] and df['RATING'].iloc[i] <= ratingq[2]:
        df['P_CLASS'].iloc[i] = 3
    if df['RATING'].iloc[i] > ratingq[2] and df['RATING'].iloc[i] <= ratingq[3]:
        df['P_CLASS'].iloc[i] = 2
    if df['RATING'].iloc[i] > ratingq[3]:
        df['P_CLASS'].iloc[i] = 1
        

In [ ]:
df = df[['DATE', 'NAME', 'RATING', 'P_CLASS', 'FROM', 'FT_RATING','T_CLASS', 'TO', 'TT_RATING',
       'FEE', 'TF_CLASS', 'POSITION', 'HEIGHT', 'FOOT', 'AGE','AGE_CLASS', 'NATION']]

# Plot player ratings vs fees for each level of combinations of from team and age classes

In [ ]:
plt.figure(figsize=(15,15))
ctr = 1

list1 = list(df['T_CLASS'].unique())
list1.sort()
for i in list1:
    if i==0:
        continue

    else:
        min_df = df[df['T_CLASS']==i]
        list2 = list(min_df['AGE_CLASS'].unique())
        list2.sort()
        for j in list2:
            newmin_df = min_df[min_df['AGE_CLASS']==j]
            plt.subplot(4, 4, ctr)
            cv = round(np.cov(newmin_df['RATING'], newmin_df['FEE'])[0, 1],2)
            plt.title('Cov(x, y) = '+str(cv))
            plt.ylim((df['FEE'].min(),df['FEE'].max()))
            plt.scatter(newmin_df['RATING'], newmin_df['FEE'], label='Team class: '+str(i)+' and age class: '+str(j))
            z = np.polyfit(newmin_df['RATING'], newmin_df['FEE'], 1)
            p = np.poly1d(z)
            intcpt = p(0)
            slope = (p(7.5)-p(6))/1.5
            plt.plot(newmin_df['RATING'], p(newmin_df['RATING']), linestyle='dashed', linewidth=1, color='red', label='y= '+str(round(slope,1))+'x + '+str(round(intcpt,1)))
            plt.legend()
            ctr+=1


plt.suptitle('Player rating (x) Vs. Transfer fee (y) for each combination of team and age classes', fontsize=20)

# Calculate causal effect of rating do(X = x) on fee Y = y for a certain class of team and age. Eg team class 1 & age class 2

In [17]:
def prob_bayes(f, a, x, y, df2):
    dff = df2[df2['T_CLASS']==f]
    dff = dff[dff['AGE_CLASS']==a]

    # P(Y)
    prob_y = len(dff[dff['TF_CLASS']==y])/len(dff)

    # P(X)
    prob_x = len(dff[dff['P_CLASS']==x])/len(dff)

    # P(X|Y)
    tr_df = dff[dff['TF_CLASS']==y]
    if len(tr_df)!= 0:
        prob_xy = len(tr_df[tr_df['P_CLASS']==x])/len(tr_df)
    else:
        prob_xy = 0
    if prob_x:
        prob = (prob_xy*prob_y)/prob_x
    else:
        prob = 0

    return prob

In [ ]:
y1_x1 = prob_bayes(1, 2, 1, 1, df)
y1_x2 = prob_bayes(1, 2, 2, 1, df)
y1_x3 = prob_bayes(1, 2, 3, 1, df)
y1_x4 = prob_bayes(1, 2, 4, 1, df)
print(y1_x1)
print(y1_x2)
print(y1_x3)
print(y1_x4)

# PDF of age from all players in top 5 league by bayesian inference

In [ ]:
dist = distfit()
dist.fit_transform(pl['AGE'])
dist.plot()

In [22]:
norm_age = np.array(pl['AGE'])
norm_age = pd.Series(norm_age)

In [ ]:
with pm.Model() as age:

  alpha = pm.HalfNormal('alpha', sd=10)
  beta = pm.HalfNormal('beta', sd=10)

  f = pm.Gamma('f', alpha=alpha, beta=beta, observed=norm_age)
  
  trace_age = pm.sample(5000)

  pm.plot_trace(trace_age, ['alpha', 'beta'], figsize=(10,7))
  plt.show()

In [ ]:
df_age = pm.summary(trace_age, kind="stats")
df_age

In [ ]:
plt.figure(figsize=(7,7))

xgam = np.linspace(10,50,100)

agam = list(df_age['mean'])[0]
bgam = list(df_age['mean'])[1]
agsd = list(df_age['sd'])[0]
bgsd = list(df_age['sd'])[1]

for a in np.linspace(agam-agsd, agam+agsd, 10):
    for b in np.linspace(bgam-bgsd, bgam+bgsd, 10):
        ygam = stats.gamma.pdf(xgam, a, scale=1/b)
        plt.plot(xgam, ygam, color='blue', alpha=0.1)

plt.xlabel('Age (years)', fontsize=15)
plt.ylabel('P(Age)', fontsize=15)
plt.title('PDF of Age', fontsize=15)
plt.show()

In [26]:
def gammafunc(x, alpha, beta):
  result = ((beta**alpha)*(x**(alpha-1))*np.exp(-1*beta*x))/gamma((alpha-1))
  return result

In [ ]:
pr1 = []
pr2 = []
pr3 = []
pr4 = []
ages = [20,25,30]

for i in range(10000):
    al = np.random.choice(np.linspace(agam-agsd, agam+agsd, 20))
    be = np.random.choice(np.linspace(bgam-bgsd, bgam+bgsd, 20))
    p_gamma = integrate.quad(gammafunc, 0, np.inf, args=(al, be))
    nc_age = p_gamma[0]
    p1 = integrate.quad(gammafunc, 0, ages[0], args=(al, be))[0]/nc_age
    pr1.append(p1)
    p2 = integrate.quad(gammafunc, ages[0], ages[1], args=(al, be))[0]/nc_age
    pr2.append(p2)
    p3 = integrate.quad(gammafunc, ages[1], ages[2], args=(al, be))[0]/nc_age
    pr3.append(p3)
    p4 = integrate.quad(gammafunc, ages[2], np.inf, args=(al, be))[0]/nc_age
    pr4.append(p4)

prob_1 = sum(pr1)/len(pr1)
prob_2 = sum(pr2)/len(pr2)
prob_3 = sum(pr3)/len(pr3)
prob_4 = sum(pr4)/len(pr4)

print("P(Age class 1) =", prob_1)
print("P(Age class 2) =", prob_2)
print("P(Age class 3) =", prob_3)
print("P(Age class 4) =", prob_4)

print("Sum of probabilities =", prob_1+prob_2+prob_3+prob_4)

In [ ]:
p_age = {1:prob_1, 2:prob_2, 3:prob_3, 4:prob_4}
p_age

# PDF of From team rating from all transfers (free+loan) in top 5 league by bayesian inference

In [29]:
df_og = pd.read_csv('alpha.csv')

trm = []
for i in range(len(df_og)):
    trm.append((df_og['FROM'].iloc[i], df_og['FT_RATING'].iloc[i]))
    trm.append((df_og['TO'].iloc[i], df_og['TT_RATING'].iloc[i]))

trm = list(set(trm))
trm = pd.DataFrame(trm, columns=['Team', 'Rating'])
trm = dict(zip(trm['Team'], trm['Rating']))

In [ ]:
zero = [0]*len(trf)
trf['FT_RATING'] = zero


for i in range(len(trf)):
    if trf['FROM'].iloc[i] in trm.keys():
        trf['FT_RATING'].iloc[i] = trm[trf['FROM'].iloc[i]]

trf = trf[['DATE',	'NAME',	'FROM', 'FT_RATING','TO',	'FEE']]
nztrf = trf[trf['FT_RATING']!=0]

dist = distfit(distr='popular')
dist.fit_transform(nztrf['FT_RATING'])
dist.plot()

In [ ]:
for_beta = np.array(nztrf['FT_RATING'])
for_beta = [(x-np.min(for_beta))/(np.max(for_beta)-np.min(for_beta)) for x in for_beta]
for_beta = [x for x in for_beta if x not in [0.0, 1.0]]

for_beta = pd.Series(for_beta)

dist = distfit(distr='popular')
dist.fit_transform(for_beta)
dist.plot()

In [ ]:
with pm.Model() as rate:

  alpha_b = pm.HalfNormal('alpha', sd=10)
  beta_b = pm.HalfNormal('beta', sd=10)

  f = pm.Beta('f', alpha=alpha_b, beta=beta_b, observed=for_beta)

  trace_team = pm.sample(5000)
  pm.plot_trace(trace_team, ['alpha', 'beta'], figsize=(10,7))

  plt.show()

In [ ]:
df_from = pm.summary(trace_team, kind="stats")
df_from

In [ ]:
plt.figure(figsize=(7,7))

xbet = np.linspace(0,1,100)

abet = list(df_from['mean'])[0]
bbet = list(df_from['mean'])[1]
absd = list(df_from['sd'])[0]
bbsd = list(df_from['sd'])[1]

for a in np.linspace(abet-absd, abet+absd, 10):
    for b in np.linspace(bbet-bbsd, bbet+bbsd, 10):
        ybet = stats.beta.pdf(xbet, a, b)
        plt.plot(xbet, ybet, color='blue', alpha=0.1)


plt.xlabel('From team rating', fontsize=15)
plt.ylabel('P(Rating)', fontsize=15)
plt.title('PDF of From team rating', fontsize=15)
plt.show()

In [35]:
def betafunc(x, alpha, beta):
  result = ((x**(alpha-1))*((1-x)**(beta-1))*gamma(alpha+beta))/(gamma(alpha)*gamma(beta))
  return result

In [ ]:
pr1 = []
pr2 = []
pr3 = []
pr4 = []

rates = fteamq[1:4]
rates = [(x-nztrf['FT_RATING'].min())/(nztrf['FT_RATING'].max()-nztrf['FT_RATING'].min()) for x in rates]

for i in range(10000):
    al = np.random.choice(np.linspace(abet-absd, abet+absd, 20))
    be = np.random.choice(np.linspace(bbet-bbsd, bbet+bbsd, 20))
    p_beta = integrate.quad(betafunc, 0, 1, args=(al, be))
    nc_rate = p_beta[0]
    p1 = integrate.quad(betafunc, 0, rates[0], args=(al, be))[0]/nc_rate
    pr1.append(p1)
    p2 = integrate.quad(betafunc, rates[0], rates[1], args=(al, be))[0]/nc_rate
    pr2.append(p2)
    p3 = integrate.quad(betafunc, rates[1], rates[2], args=(al, be))[0]/nc_rate
    pr3.append(p3)
    p4 = integrate.quad(betafunc, rates[2], 1, args=(al, be))[0]/nc_rate
    pr4.append(p4)

prob_1 = sum(pr1)/len(pr1)
prob_2 = sum(pr2)/len(pr2)
prob_3 = sum(pr3)/len(pr3)
prob_4 = sum(pr4)/len(pr4)

print("P(Team class 1) =", prob_4)
print("P(Team class 2) =", prob_3)
print("P(Team class 3) =", prob_2)
print("P(Team class 4) =", prob_1)

print("Sum of probabilities =", prob_1+prob_2+prob_3+prob_4)

In [ ]:
p_rate = {4:prob_1, 3:prob_2, 2:prob_3, 1:prob_4}
p_rate

# Average causal effect of do(X=x) on Y=y over all levels

In [38]:
def avg_prob(dfa, p_rate, p_age):
  tr_ot_dict = {}
  for t in range(1,5):
    out = []
    for o in range(1,5):
      p_avg = 0
      for i in range(1,5):
        for j in range(1,5):
          ps = prob_bayes(i, j, t, o, dfa)
          p_avg+=(ps*p_rate[i]*p_age[j])
        
      out.append(round(p_avg,5))
    tr_ot_dict[t] = out

  return tr_ot_dict

In [39]:
total_effect = avg_prob(df,p_rate,p_age)
tot_eff_df = pd.DataFrame.from_dict(total_effect, orient='index', columns=[1,2,3,4])

In [ ]:
total_effect

In [ ]:
tot_eff_df = tot_eff_df.rename(columns={1: 'Fee class 1', 2: 'Fee class 2', 3: 'Fee class 3', 4: 'Fee class 4'}, index={1: 'Player class 1', 2: 'Player class 2', 3: 'Player class 3', 4: 'Player class 4'})
tot_eff_df

In [110]:
def abc(dfs, p_rate, p_age):
    p1 = 0
    p2 = 0
    ran1 = list(set(dfs['T_CLASS'].unique()))
    for i in ran1:
        
        dffs = dfs[dfs['T_CLASS']==i]
        ran2 = list(set(dffs['AGE_CLASS'].unique()))
        for j in ran2:
            dframe = dffs[dffs['AGE_CLASS']==j]
            for k in range(len(dframe)):
                if dframe['P_CLASS'].iloc[k]!=1:
                    dframe['P_CLASS'].iloc[k] = 0
            # p(Y=1)
            prob_y = len(dframe[dframe['TF_CLASS']==1])/len(dframe)

            # P(X=1)
            prob_x1 = len(dframe[dframe['P_CLASS']==1])/len(dframe)

            # p(X=0)
            prob_x0 = len(dframe[dframe['P_CLASS']==0])/len(dframe)


            # P(X1|Y)
            tr_df1 = dframe[dframe['TF_CLASS']==1]
            if len(tr_df1)!= 0:
                prob_x1y = len(tr_df1[tr_df1['P_CLASS']==1])/len(tr_df1)
            else:
                prob_x1y = 0

            # P(X0|Y)
            tr_df2 = dframe[dframe['TF_CLASS']==1]
            if len(tr_df2)!= 0:
                prob_x0y = len(tr_df2[tr_df2['P_CLASS']==0])/len(tr_df2)
            else:
                prob_x0y = 0


            if prob_x1:
                prob1 = (prob_x1y*prob_y)/prob_x1
            else:
                prob1 = 0


            if prob_x0:
                prob2 = (prob_x0y*prob_y)/prob_x0
            else:
                prob2 = 0

            p1+=prob1*p_rate[i]*p_age[j]
            p2+=prob2*p_rate[i]*p_age[j]
    
    
    return p1, p2

In [ ]:
pro1 = []
pro2 = []
for i in range(5000):
    d = df.sample(n=400)
    pr1, pr2 = abc(d, p_rate, p_age)
    pro1.append(pr1)
    pro2.append(pr2)

diff = [pro1[i]-pro2[i] for i in range(len(pro1))]

plt.hist(diff, bins=50)
plt.show()

In [112]:
from statsmodels.stats.weightstats import ztest

In [113]:
ztest_Score, p_value= ztest(diff,value = 0.0)

In [ ]:
x_axis = [1, 2, 3, 4]
teamcls = [0.043318535641380766, 0.08317465649507239, 0.045299583949734455, 0.018092638556812226]
agecls = [0.02159810139819831, 0.11451391166403328, 0.04685361731451872, 0]
plt.plot(x_axis, teamcls, color='blue', label = 'Team class')
plt.plot(x_axis, agecls, color='red', label='Age class')
plt.xlabel('class number')
plt.ylabel('P(y=1|x=1)-P(y=1|x=~1)')
plt.legend()
plt.show()